In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [ ]:
import grader

# SQL Miniproject

## Introduction

The city of New York does restaurant inspections and assigns a grade. Inspections data for the last 4 years are available on s3 [here](s3://dataincubator-course/coursedata/nyc_inspection_data.zip). You can copy it from s3 with this command:

`!aws s3 cp s3://dataincubator-course/coursedata/nyc_inspection_data.zip .`

The file `RI_Webextract_BigApps_Latest.xls` contains a description of each of the datafiles.  Take a look and then load the csv formatted `*.txt` files into
a database as five tables:
1. `actions`
2. `cuisines`
3. `violations`
4. `grades` (from `WebExtract.txt`)
5. `boroughs` (from `RI_Webextract_BigApps_Latest.xls`)

## SQLite3

The project should be written in SQL. Between SQLite and PostgreSQL we recommend sqlite3 for this project.  You can use the sqlite command prompt by running this command in bash
```bash
sqlite3 cmd "DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"
```
Alternatively, you can run bash commands in a jupyter notebook by prepending the `!` in a code cell (notice that we conveniently get the output displayed

In [ ]:
!sqlite3 cmd """\
DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"""

Finally, we use the [ipython-sql extension](https://github.com/catherinedevlin/ipython-sql#ipython-sql) by first loaidng the sql extension and then running our code with the "magic" command in the first line
```python
%%sql sqlite://
```
Notice that the output table is formatted nicely as a nice HTML table.

In [ ]:
%load_ext sql

In [ ]:
%%sql sqlite://
DROP TABLE IF EXISTS writer;
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);

SELECT * FROM writer;

## Loading data


The Sqlite3 has a convenient [`.import` function](https://sqlite.org/cli.html#csv_import) which can create tables from `.csv` files.

```bash
sqlite> .import sample.csv.nogit sample
sqlite> SELECT * FROM sample;
```

The files may contain malformatted text.  Unfortunately, this is all too common.  As a stop gap, remember that [`iconv`](https://linux.die.net/man/1/iconv) is a unix utility that can convert files between different text encodings.

Alternatively, you can also read csv files using pandas and convert that into SQL via some sql magic.

Also be aware that the `WebExtract.txt` file contains duplicated data. Multiple rows with identical `CAMIS` and `INSPDATE` values should be reduced to a single row. You will need the `SCORE`, `ZIPCODE`, `BORO`, and `CURRENTGRADE` columns for this miniproject. Make sure that you use a non-null value from the multiple rows for each of these columns when reducing to a single row.

In [ ]:
!printf "Name,Age\nAlice,3\nBob,10" > sample.csv.nogit

In [ ]:
import pandas as pd
sample = pd.read_csv('sample.csv.nogit')
%sql DROP TABLE IF EXISTS sample
%sql PERSIST sample
%sql SELECT * FROM sample;

## Question 1: null_entries

Return the number of entries in the grades table with a blank score. Remove those rows from the dataset for the rest of the questions in the assignment.

**Question:** How else might we have handled this?

In [ ]:
def null_entries():
    return 0

grader.score('sql__null_entries', null_entries)

## Question 2: score_by_zipcode

Return a list of tuples of the form:

    (zipcode, mean score, number of restaurants)

for each of the 92 zipcodes in the city with over 100 restaurants. Use the score from the latest inspection date for each restaurant. Sort the list in ascending order by mean score.

**Note:** There is an interesting discussion here about what the mean score *means* in this dataset. Think about what we're actually calculating - does it represent what we're trying to understand about these zipcodes?

What if we use the average of a restaurant's inspections instead of the latest?

**Checkpoints:**
- Total unique restaurants: 25,232;
- Total restaurants in valid zipcodes: 20,349

In [ ]:
def score_by_zipcode():
    return [("11201", 9.81739130434783, 345)] * 92

grader.score('sql__score_by_zipcode', score_by_zipcode)

## Question 3: score_by_map

The above are not terribly enlightening.  Use [CartoDB](http://cartodb.com/) to produce a map of average scores by zip code.  You can sign up for a free trial.

You will have to use their wizard to plot the data by [zipcode](https://carto.com/learn/guides/analysis/georeference). You will need to specify "USA" in the countryfield.  Then use the "share" button to return a link of the form [https://x.cartodb.com/](https://x.cartodb.com/).

**For fun:** How do JFK, Brighton Beach, Liberty Island (home of the Statue of Liberty), Financial District, Chinatown, and Coney Island fare?

**For more fun:** Plot restaurants as pins on the map, allowing the user to filter by "low", "middling", or "high"-scoring restaurants. You can use a CASE WHEN statement to create the different groups based on score thresholds.

In [ ]:
def score_by_map():
    # must be url of the form https://x.cartodb.com/...
    return "https://cartodb.com"

grader.score('sql__score_by_map', score_by_map)

## Question 4: score_by_borough
Return a list of tuples of the form:

    (borough, mean score, number of restaurants)

for each of the city's five boroughs. Sort the list in ascending order by grade.

**Hint:** You will have to perform a join with the `boroughs` table. The borough names should be reported in ALL CAPS.

**Checkpoint:**
- Total restaurants in valid boroughs: 25,220

In [ ]:
def score_by_borough():
    return [("MANHATTAN", 10.7269875502402, 10201)] * 5

grader.score('sql__score_by_borough', score_by_borough)

## Question 5: score_by_cuisine

Return a list of the 75 tuples of the form

    (cuisine, mean score, number of reports)

for each of the 75 cuisine types with at least 100 violation reports. Sort the list in ascending order by score. Are the least sanitary and most sanitary
cuisine types surprising?

**Note:** It's interesting to think again about what this analysis is trying to say and how it differs from the analysis by zipcode. How should this
affect the calculation in your opinion?

**Checkpoint:**
- Total entries from valid cuisines: 531,529

In [ ]:
def score_by_cuisine():
    return [("French", 20.3550686378036, 7576)] * 75

grader.score('sql__score_by_cuisine', score_by_cuisine)

## Question 6: violation_by_cuisine
Which cuisines tend to have a disproportionate number of what which violations? Answering this question isn't easy becuase you have to think carefully about normalizations.

1. More popular cuisine categories will tend to have more violations just becuase they represent more restaurants.
2. Similarly, some violations are more common.  For example, knowing that "Equipment not easily movable or sealed to floor" is a common violation for Chinese restuarants is not particularly helpful when it is a common violation for all restaurants.

The right quantity is to look at is the conditional probability of a specific type of violation given a specific cuisine type and divide it by the unconditional probability of the violation for the entire population. Taking this ratio gives the right answer.  Return the 20 highest ratios of the form:

    ((cuisine, violation), ratio, count)

**Hint:**
1. You might want to check out this [Stackoverflow post](http://stackoverflow.com/questions/972877/calculate-frequency-using-sql).
2. The definition of a violation changes with time.  For example, 10A can mean two different things "Toilet facility not maintained ..." or "Vermin or other live animal present ..." when things were prior to 2003. To deal with this, you should limit your analysis to violation codes with end date after Jan 1, 2014. (This end date refers to the validity time ranges in Violation.txt).
3. The ratios don't mean much when the number of violations of a given type and for a specific category are not large (why not?).  Be sure to filter these out.  We chose 100 as our cutoff.

**Checkpoint:**
- Top 20 ratios mean: 2.37009216349859

In [ ]:
def violation_by_cuisine():
    return [(("Café/Coffee/Tea",
              "Toilet facility not maintained and provided with toilet paper; "
              "waste receptacle and self-closing door."),
             1.87684775827172, 315)] * 20

grader.score('sql__violation_by_cuisine', violation_by_cuisine)

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*